# NBA data reports

In [0]:
import pyspark

### Loading data for NBA ranking

In [0]:
# File location and type
file_location = "/FileStore/tables/nba_ranking.csv"
file_type = "csv"

nba_ranking_df = spark.read.format(file_type) \
  .option("inferSchema", "false") \
  .option("header", "true") \
  .option("sep", ",") \
  .load(file_location)

nba_ranking_df.createOrReplaceTempView('nba_ranking_df')
#nba_ranking_df.show(2)

### Loading data for NBA teams

In [0]:
# File location and type
file_location = "/FileStore/tables/nba_teams.csv"
file_type = "csv"

nba_teams_df = spark.read.format(file_type) \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", ",") \
  .load(file_location)


# Add a column with full team name for NBA teams DataFrame
from pyspark.sql.functions import lit, concat, col

nba_teams_df = nba_teams_df.withColumn('TEAM_NAME', concat(col('CITY'), lit(' '), col('NICKNAME')))\
                           .withColumnRenamed('NICKNAME', 'TEAM_NICKNAME')

nba_teams_df.createOrReplaceTempView('nba_teams_df')
#nba_teams_df.show(2)

### Loading data for NBA game details

In [0]:
# File location and type
file_location = "/FileStore/tables/nba_games_details.csv"
file_type = "csv"

nba_games_details_df = spark.read.format(file_type) \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", ",") \
  .load(file_location)


nba_games_details_df = nba_games_details_df.withColumnRenamed('NICKNAME', 'PLAYER_NICKNAME')

#nba_games_details_df.createOrReplaceTempView('nba_games_details_df')
#nba_games_details_df.show(2)

### Loading data for NBA games

In [0]:
# File location and type
file_location = "/FileStore/tables/nba_games.csv"
file_type = "csv"

nba_games_df = spark.read.format(file_type) \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", ",") \
  .load(file_location)


#nba_games_df.createOrReplaceTempView('nba_games_df')
#nba_games_df.show(2)

### Loading data for NBA players

In [0]:
# File location and type
file_location = "/FileStore/tables/nba_players.csv"
file_type = "csv"

nba_players_df = spark.read.format(file_type) \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", ",") \
  .load(file_location)


#nba_players_df.createOrReplaceTempView('nba_players_df')
#nba_players_df.show(2)

### Modify raw rawdata to present: 
1. stats of players in every game;
2. final scores of each game;
3. team ID's of home & visitor team

In [0]:
list_of_columns = ['SEASON',
 nba_games_details_df['GAME_ID'],
 nba_games_details_df['TEAM_ID'],
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'PLAYER_ID',
 'PLAYER_NAME',
 'PLAYER_NICKNAME',
 'START_POSITION',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TO',
 'PF',
 'PTS',
 'PLUS_MINUS',
 'PTS_HOME', 
 'PTS_AWAY',
 'HOME_TEAM_ID', 
 'VISITOR_TEAM_ID',
 'GAME_DATE_EST']

In [0]:
# NAJLEPSZA WERSJA SO FAR
game_stats_pre_final = nba_games_details_df.join(nba_teams_df, nba_games_details_df['TEAM_ID'] == nba_teams_df['TEAM_ID'], 'INNER')\
                                           .join(nba_games_df, nba_games_details_df['GAME_ID'] == nba_games_df['GAME_ID'], 'INNER')\
                                           .select(list_of_columns)
#display(game_stats_pre_final)

In [0]:
#game_stats_pre_final.write.mode('overwrite').parquet('/FileStore/powerBi/game_stats_pre_final.parquet')

### Creating a view in order to get the data in SQL format in advance

In [0]:
game_stats_pre_final.createOrReplaceTempView('game_stats_pre_final')

### Show players who scored the most points in last 10 seasons since 2020 and rank them points descending (include their team at that time)

In [0]:
%sql
SELECT SEASON, TEAM_NAME, PLAYER_NAME, PTS, RANK 
FROM (
  SELECT SEASON, 
         TEAM_NAME,
         PLAYER_NAME, 
         PTS, 
         COUNT(PLAYER_NAME) AS NAME_COUNT,
         ROW_NUMBER() OVER(ORDER BY PTS DESC) AS RANK
  FROM game_stats_pre_final A 
  WHERE SEASON BETWEEN 2011 AND 2020
  AND PTS = (SELECT MAX(PTS) 
         FROM game_stats_pre_final
         WHERE SEASON = A.SEASON)
  GROUP BY SEASON,TEAM_NAME, PLAYER_NAME, PTS
  HAVING NAME_COUNT >= 1
)

SEASON,TEAM_NAME,PLAYER_NAME,PTS,RANK
2016,Phoenix Suns,Devin Booker,70.0,1
2020,Golden State Warriors,Stephen Curry,62.0,2
2013,New York Knicks,Carmelo Anthony,62.0,3
2019,Portland Trail Blazers,Damian Lillard,61.0,4
2018,Houston Rockets,James Harden,61.0,5
2015,Los Angeles Lakers,Kobe Bryant,60.0,6
2017,Houston Rockets,James Harden,60.0,7
2011,Brooklyn Nets,Deron Williams,57.0,8
2014,Cleveland Cavaliers,Kyrie Irving,57.0,9
2012,Golden State Warriors,Stephen Curry,54.0,10


### Find most efficient player in the league in terms of 3 points made (shot %) - Prepare a chart

In [0]:
%sql
SELECT PLAYER_NAME,
       SUM(FG3A) AS SHOT_ATTEMPTS, 
       SUM(FG3M) AS SHOTS_MADE, 
       ROUND(AVG(FG3_PCT), 3) AS SHOT_PRCNTG
FROM game_stats_pre_final
WHERE SEASON BETWEEN 2014 AND 2020
GROUP BY PLAYER_NAME
HAVING SUM(FG3A) > 2000 -- considering only these players who threw at least 2000 balls from 3 point line
ORDER BY SHOT_PRCNTG DESC

PLAYER_NAME,SHOT_ATTEMPTS,SHOTS_MADE,SHOT_PRCNTG
Stephen Curry,5847.0,2486.0,0.42
JJ Redick,3441.0,1466.0,0.42
Klay Thompson,3928.0,1666.0,0.416
Kyle Korver,2702.0,1158.0,0.409
Kevin Durant,2555.0,1002.0,0.403
Joe Harris,2181.0,941.0,0.397
Buddy Hield,3082.0,1237.0,0.393
Kyrie Irving,2952.0,1169.0,0.388
Khris Middleton,2878.0,1134.0,0.384
Paul George,3838.0,1496.0,0.383


Output can only be rendered in Databricks

### Display and compare highest TOP 5 scores of Golden State Warriors with Cleveland Cavaliers in 2014 (HOME & AWAY)

- Top 5 scores at HOME

In [0]:
%sql
SELECT * FROM (
  SELECT game_stats_pre_final.GAME_ID,
         game_stats_pre_final.TEAM_ABBREVIATION,
         game_stats_pre_final.TEAM_NAME AS HOME_TEAM, 
         game_stats_pre_final.PTS_HOME, 
         game_stats_pre_final.PTS_AWAY, 
         --game_stats_pre_final.VISITOR_TEAM_ID,
         NBA_TEAMS_DF.TEAM_NAME AS VS_OPPONENT,
         ROW_NUMBER() OVER(PARTITION BY TEAM_ABBREVIATION ORDER BY PTS_HOME DESC) SERIES
    FROM game_stats_pre_final
    INNER JOIN NBA_TEAMS_DF ON game_stats_pre_final.VISITOR_TEAM_ID = NBA_TEAMS_DF.TEAM_ID
    WHERE game_stats_pre_final.TEAM_ID = game_stats_pre_final.HOME_TEAM_ID -- games at home only
    AND game_stats_pre_final.TEAM_ABBREVIATION IN ('GSW', 'CLE') 
    AND game_stats_pre_final.SEASON = 2014
    GROUP BY game_stats_pre_final.GAME_ID, game_stats_pre_final.TEAM_ABBREVIATION, game_stats_pre_final.TEAM_NAME, game_stats_pre_final.PTS_HOME, game_stats_pre_final.PTS_AWAY, game_stats_pre_final.VISITOR_TEAM_ID,     NBA_TEAMS_DF.TEAM_NAME
    ORDER BY game_stats_pre_final.PTS_HOME DESC
    
)
WHERE SERIES <= 5

GAME_ID,TEAM_ABBREVIATION,HOME_TEAM,PTS_HOME,PTS_AWAY,VS_OPPONENT,SERIES
21401229,GSW,Golden State Warriors,133.0,126.0,Denver Nuggets,1
21400643,CLE,Cleveland Cavaliers,129.0,90.0,Charlotte Hornets,1
21400741,GSW,Golden State Warriors,128.0,114.0,Dallas Mavericks,2
21400415,GSW,Golden State Warriors,128.0,108.0,Sacramento Kings,3
21400133,CLE,Cleveland Cavaliers,127.0,94.0,Atlanta Hawks,2
21400038,GSW,Golden State Warriors,127.0,104.0,Los Angeles Lakers,4
21400493,GSW,Golden State Warriors,126.0,105.0,Toronto Raptors,5
21400416,CLE,Cleveland Cavaliers,125.0,104.0,Minnesota Timberwolves,3
21400766,CLE,Cleveland Cavaliers,120.0,105.0,Los Angeles Lakers,4
41400304,CLE,Cleveland Cavaliers,118.0,88.0,Atlanta Hawks,5


- Top 5 scores AWAY

In [0]:
%sql
SELECT * FROM (
  SELECT game_stats_pre_final.GAME_ID,
         game_stats_pre_final.TEAM_ABBREVIATION,
         game_stats_pre_final.TEAM_NAME AS HOME_TEAM, 
         game_stats_pre_final.PTS_HOME, 
         game_stats_pre_final.PTS_AWAY, 
         NBA_TEAMS_DF.TEAM_NAME AS VS_OPPONENT,
         ROW_NUMBER() OVER(PARTITION BY TEAM_ABBREVIATION ORDER BY PTS_HOME DESC) SERIES
    FROM game_stats_pre_final
    INNER JOIN NBA_TEAMS_DF ON game_stats_pre_final.HOME_TEAM_ID = NBA_TEAMS_DF.TEAM_ID
    WHERE game_stats_pre_final.TEAM_ID = game_stats_pre_final.VISITOR_TEAM_ID -- games away only
    AND game_stats_pre_final.TEAM_ABBREVIATION IN ('GSW', 'CLE') 
    AND game_stats_pre_final.SEASON = 2014
    GROUP BY game_stats_pre_final.GAME_ID, game_stats_pre_final.TEAM_ABBREVIATION, game_stats_pre_final.TEAM_NAME, game_stats_pre_final.PTS_HOME, game_stats_pre_final.PTS_AWAY, game_stats_pre_final.VISITOR_TEAM_ID,     NBA_TEAMS_DF.TEAM_NAME
    ORDER BY game_stats_pre_final.PTS_HOME DESC
)
WHERE SERIES <= 5

GAME_ID,TEAM_ABBREVIATION,HOME_TEAM,PTS_HOME,PTS_AWAY,VS_OPPONENT,SERIES
41400314,GSW,Golden State Warriors,128.0,115.0,Houston Rockets,1
21400593,GSW,Golden State Warriors,127.0,115.0,Oklahoma City Thunder,2
21400964,CLE,Cleveland Cavaliers,125.0,128.0,San Antonio Spurs,1
21400749,GSW,Golden State Warriors,124.0,116.0,Atlanta Hawks,3
21400348,GSW,Golden State Warriors,122.0,128.0,New Orleans Pelicans,4
21400599,CLE,Cleveland Cavaliers,121.0,126.0,Los Angeles Clippers,2
21400125,CLE,Cleveland Cavaliers,121.0,122.0,Boston Celtics,3
21400336,CLE,Cleveland Cavaliers,119.0,114.0,New Orleans Pelicans,4
11400033,CLE,Cleveland Cavaliers,119.0,122.0,Miami Heat,5
41400143,GSW,Golden State Warriors,119.0,123.0,New Orleans Pelicans,5


###  Present top 10 most efficient players in terms of average points per game found in data set

In [0]:
%sql
SELECT SEASON,
       PLAYER_NAME,
       ROUND(AVG(PTS), 2) AS AVG_POINTS
FROM   game_stats_pre_final
GROUP BY SEASON, PLAYER_NAME
ORDER BY AVG_POINTS DESC
LIMIT 10

SEASON,PLAYER_NAME,AVG_POINTS
2018,James Harden,34.91
2005,Kobe Bryant,33.74
2019,James Harden,33.6
2005,Allen Iverson,32.8
2020,Stephen Curry,31.51
2022,Luka Doncic,31.42
2006,Kobe Bryant,31.25
2016,Russell Westbrook,31.24
2022,Shai Gilgeous-Alexander,31.17
2022,Joel Embiid,31.13


### Present top 3 coaches who had most wins during the regular season for 2014 and rank them

In [0]:
%sql
SELECT SEASON_ID, HEADCOACH, G, W, L, TEAM_NAME, CONFERENCE, ROW_NUMBER() OVER(ORDER BY W DESC) AS RANKING
FROM NBA_RANKING_DF R
INNER JOIN NBA_TEAMS_DF T 
        ON R.TEAM_ID = T.TEAM_ID  
WHERE G = 82
AND SEASON_ID = 22014
GROUP BY SEASON_ID, HEADCOACH, G, W, L, TEAM_NAME, CONFERENCE
ORDER BY W DESC
LIMIT 3

SEASON_ID,HEADCOACH,G,W,L,TEAM_NAME,CONFERENCE,RANKING
22014,Steve Kerr,82,67,15,Golden State Warriors,West,1
22014,Lloyd Pierce,82,60,22,Atlanta Hawks,East,2
22014,Doc Rivers,82,56,26,Los Angeles Clippers,West,3


### Show stats of 4 random players for 2014

In [0]:
%sql
SELECT TEAM_NAME, 
       PLAYER_NAME, 
       FLOOR(ROUND(AVG(PTS))) AS AVG_PTS,
       FLOOR(ROUND(AVG(AST))) AS AVG_AST,
       FLOOR(ROUND(AVG(REB))) AS AVG_REB,
       FLOOR(ROUND(AVG(BLK))) AS AVG_BLK,
       FLOOR(ROUND(AVG(FGM))) AS AVG_FGM,
       FLOOR(ROUND(AVG(FG3M))) AS AVG_FG3M
FROM game_stats_pre_final
WHERE PLAYER_NAME IN ('Stephen Curry', 'LeBron James', 'Draymond Green', 'Chris Paul')
AND SEASON = 2014
GROUP BY TEAM_NAME, PLAYER_NAME


TEAM_NAME,PLAYER_NAME,AVG_PTS,AVG_AST,AVG_REB,AVG_BLK,AVG_FGM,AVG_FG3M
Los Angeles Clippers,Chris Paul,19,10,4,0,7,2
Golden State Warriors,Draymond Green,12,4,8,1,4,1
Golden State Warriors,Stephen Curry,24,7,4,0,8,4
Cleveland Cavaliers,LeBron James,26,8,7,1,9,2
